<a href="https://colab.research.google.com/github/SeyfGoumeida/NLP-world-embadding-/blob/main/NLP(world_embadding).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install transformers


In [37]:
from time import time 
import pandas as pd
import numpy as np
from transformers import BertModel,BertTokenizer
import torch

In [38]:
model_name = "bert-base-cased"


In [39]:
if torch.cuda.is_available():
  device ='cuda'
else:
  device = 'cpu'

In [40]:
model = BertModel.from_pretrained(model_name,output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:
if device =='cuda':
  model.cuda()

In [42]:
sample ="let's encode a sentence"
print('raw : ',sample)
tokenized = tokenizer.tokenize(sample)[:512]#taille max des senteces en bert
print('Tokenized :',tokenized)
input_inds = tokenizer.convert_tokens_to_ids(tokenized)
print('encoded :',input_inds)

raw :  let's encode a sentence
Tokenized : ['let', "'", 's', 'en', '##code', 'a', 'sentence']
encoded : [1519, 112, 188, 4035, 13775, 170, 5650]


In [43]:
tensor_sentence = torch.tensor([input_inds])
tensor_sentence = tensor_sentence.to(device)
tensor_sentence

tensor([[ 1519,   112,   188,  4035, 13775,   170,  5650]], device='cuda:0')

In [44]:
output= model(tensor_sentence)
hidden_states = output.hidden_states
len(hidden_states)

13

In [45]:
hidden_states = hidden_states[1:]
len(hidden_states)

12

In [46]:
hidden_states[0][0].shape

torch.Size([7, 768])

In [47]:
word_embadding = [layer_embadding[0].detach().cpu().numpy() for layer_embadding in hidden_states]

In [48]:
document_embadding = [layer_word_embedding.mean(axis=0) for layer_word_embedding in word_embadding]

In [49]:
len(document_embadding)

12

In [50]:
document_embadding[0].shape

(768,)

In [53]:
def encode_sentence(sentence,model, tokenizer):  
  tokenized = tokenizer.tokenize(sentence)[:512]
  input_inds = tokenizer.convert_tokens_to_ids(tokenized)
  
  tensor_sentence = torch.tensor([input_inds])
  tensor_sentence = tensor_sentence.to(device)
  
  output = model(tensor_sentence)
  hidden_states = output.hidden_states
  
  hidden_states = hidden_states[1:]
  
  document_embeddings = [layer_embeddings[0].mean(axis=0).detach().cpu().numpy() for layer_embeddings in hidden_states]
  
  return document_embeddings

In [54]:
document_embaddings = encode_sentence(sample,model,tokenizer)

In [55]:
len(document_embaddings)


12

In [56]:
sentences =["let's encode sentences","me first","each in turn"]

In [57]:
encoded_sentences = []
for sentence in sentences :
  document_embeddings = encode_sentence(sentence,model,tokenizer)
  document_embeddings = np.vstack(document_embeddings)
  encoded_sentences.append(document_embeddings)

In [58]:
len(encoded_sentences)

3

In [59]:
concat_embeddings = np.array(encoded_sentences)
concat_embeddings.shape

(3, 12, 768)

In [60]:
final_embeddings = np.swapaxes(concat_embeddings,0,1)
final_embeddings.shape

(12, 3, 768)

In [62]:
martices = list(final_embeddings)
len(martices)


12

In [63]:
martices[0].shape

(3, 768)

# Import Data


In [67]:
url ="https://drive.google.com/uc?id=1SpcExC4Fs2H-_9hdem4uz_6bd8L12RgF"
df = pd.read_csv(url,index_col = 0)

In [68]:
df

,text,label
0,Milestones in Cataloging In the case of the pr...,cisi
1,childhood psychosis. a description is given of...,med
2,neonatal hepatitis or familial neonatal obstru...,med
3,Handbook of Comparative Librarianship The firs...,cisi
4,Design and Evaluation of Information Systems T...,cisi
...,...,...
3886,modification of autistic behavior with lsd-25....,med
3887,Patterns of Evaluation in Science: Institution...,cisi
3888,The government of the American Public Library ...,cisi
3889,base pressure at subsonic speeds in the presen...,cran


In [71]:
model_name = 'bert-base-cased'
texts = df['text'].values
labels = df['label'].values

In [78]:
def encode_sentences(model,tokenizer,sentences):
  encoded_sentences = []
  for sentence in sentences:
    document_embeddings = encode_sentence(sentence,model,tokenizer)
    document_embeddings = np.vstack(document_embeddings)
    encoded_sentences.append(document_embeddings)
  concat_embeddings = np.array(encoded_sentences)
  final_embeddings = np.swapaxes(concat_embeddings, 0, 1)
  matrices = list(final_embeddings)
  return matrices

In [79]:
s =time()
print(device)
matrices = encode_sentences(model,tokenizer,texts)
print(time() - s)

cuda
165.54844069480896
